# استخدام RAG مع خدمة Azure AI Agent و Semantic Kernel

يُظهر هذا المقتطف البرمجي كيفية إنشاء وإدارة وكيل Azure AI لتوليد المحتوى المعزز بالاسترجاع (RAG) باستخدام `Azure AI Agent Service` و `Semantic Kernel`. يقوم الوكيل بمعالجة استفسارات المستخدم بناءً على السياق المسترجع ويقدم ردودًا دقيقة وفقًا لذلك.


إصلاح إصدار SQLite  
إذا واجهت الخطأ:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

قم بإلغاء تعليق هذا الجزء من الكود في بداية دفتر الملاحظات الخاص بك:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### استيراد الحزم  
يستورد الكود التالي الحزم اللازمة:  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# استرجاع النصوص المعززة بالتوليد باستخدام Semantic Kernel وخدمة Azure AI Agent

يوضح هذا المثال كيفية استخدام **خدمة Azure AI Agent** لتنفيذ **التوليد المعزز بالاسترجاع (RAG)** من خلال دمج نموذج لغوي مع سياق محدد من مجال معين مأخوذ من مستند تم تحميله.

### كيفية عمل النظام

1. **تحميل المستند**: يتم تحميل ملف Markdown (document.md) يحتوي على معلومات (سياسة تأمين السفر الخاصة بـ Contoso) إلى خدمة الوكيل.

2. **إنشاء مخزن المتجهات**: يتم فهرسة المستند في مخزن متجهات لتمكين البحث الدلالي في محتوياته.

3. **تكوين الوكيل**: يتم إنشاء وكيل باستخدام نموذج `gpt-4o` مع التعليمات الصارمة التالية:
   - الإجابة فقط على الأسئلة بناءً على المحتوى المسترجع من المستند.
   - الامتناع عن الإجابة إذا كان السؤال خارج نطاق المستند.

4. **دمج أداة البحث في الملفات**: يتم تسجيل أداة `FileSearchTool` مع الوكيل، مما يتيح للنموذج البحث واسترجاع المقاطع ذات الصلة من المستند المفهرس أثناء عملية الاستدلال.

5. **تفاعل المستخدم**: يمكن للمستخدمين طرح الأسئلة. إذا تم العثور على معلومات ذات صلة في المستند، يقوم الوكيل بتوليد إجابة مستندة إلى المحتوى.  
   وإذا لم يتم العثور على معلومات كافية، يرد الوكيل بشكل صريح بأن المستند لا يحتوي على معلومات كافية.


### الوظيفة الرئيسية

```python
# هذه هي النقطة التي يبدأ فيها البرنامج التنفيذ
def main():
    # استدعاء وظيفة الإعداد
    setup_environment()

    # تنفيذ العملية الرئيسية
    result = perform_main_task()

    # عرض النتيجة للمستخدم
    print("النتيجة النهائية:", result)
```

### الإعداد

```python
# تهيئة البيئة قبل بدء التنفيذ
def setup_environment():
    # إعداد المتغيرات الأساسية
    initialize_variables()

    # التحقق من المتطلبات
    check_prerequisites()
```

### العملية الرئيسية

```python
# تنفيذ المهمة الرئيسية للبرنامج
def perform_main_task():
    # تنفيذ الخطوة الأولى
    step_one_result = execute_step_one()

    # تنفيذ الخطوة الثانية باستخدام نتيجة الخطوة الأولى
    step_two_result = execute_step_two(step_one_result)

    # إرجاع النتيجة النهائية
    return step_two_result
```

### خطوات التنفيذ

```python
# تنفيذ الخطوة الأولى
def execute_step_one():
    # منطق الخطوة الأولى
    return "نتيجة الخطوة الأولى"

# تنفيذ الخطوة الثانية
def execute_step_two(input_data):
    # منطق الخطوة الثانية باستخدام البيانات المدخلة
    return f"نتيجة الخطوة الثانية بناءً على {input_data}"
```


تأكد من تشغيل الأمر `az login` أولاً باستخدام Azure CLI لضمان توفير سياق المصادقة الصحيح أثناء استخدام `DefaultAzureCredential`. لا تستخدم خدمة Azure AI Agent Service مفاتيح API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```

# مقدمة

مرحبًا بك في دليلنا! في هذا المستند، سنغطي بعض المفاهيم الأساسية التي تحتاج إلى معرفتها للبدء.

## المتطلبات

قبل أن تبدأ، تأكد من أن لديك ما يلي:

- حساب مفعّل
- الوصول إلى الإنترنت
- الأدوات اللازمة مثبتة على جهازك

[!NOTE] تأكد من تحديث جميع البرامج إلى أحدث إصدار.

## الخطوات الأولى

### الخطوة 1: إنشاء مشروع جديد

لبدء مشروع جديد، اتبع الخطوات التالية:

1. افتح التطبيق.
2. انقر على "ملف" > "مشروع جديد".
3. أدخل اسم المشروع واختر الموقع.
4. اضغط على "إنشاء".

[!TIP] يمكنك استخدام القوالب لتسريع عملية الإعداد.

### الخطوة 2: إعداد البيئة

بعد إنشاء المشروع، قم بإعداد البيئة:

- تأكد من أن جميع المتغيرات البيئية مضبوطة بشكل صحيح.
- تحقق من أن جميع المكتبات المطلوبة مثبتة.

[!IMPORTANT] لا تنسَ مراجعة إعدادات الأمان.

### الخطوة 3: تشغيل المشروع

لتشغيل المشروع، استخدم الأمر التالي:

@@INLINE_CODE_0@@

[!WARNING] تأكد من تشغيل الأمر في الدليل الصحيح.

## المشاكل الشائعة

### المشكلة: خطأ في الاتصال

إذا واجهت خطأ في الاتصال، تحقق مما يلي:

- هل الاتصال بالإنترنت مستقر؟
- هل إعدادات الخادم صحيحة؟

[!CAUTION] لا تقم بمشاركة بيانات الاعتماد الخاصة بك مع أي شخص.

### المشكلة: مكتبة مفقودة

إذا ظهرت رسالة خطأ تشير إلى مكتبة مفقودة، قم بتثبيتها باستخدام:

@@INLINE_CODE_1@@

[!TIP] تحقق من التوثيق لمعرفة الإصدار المطلوب.

## الخاتمة

نأمل أن يكون هذا الدليل قد ساعدك في البدء. إذا كانت لديك أي أسئلة، فلا تتردد في التواصل معنا.

شكراً لاستخدامك دليلنا!



---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
